## Questions to Answer
        
        ## 1.Which brewery produces the strongest beers by ABV%?
        ## 2.If you had to pick 3 beers to recommend using only this data, which would you pick?
## 3. Which of the factors (aroma, taste, appearance, palette) are most important in determining the overall quality of a beer?
        ## 4.Lastly, if I typically enjoy a beer due to its aroma and appearance, which beer style should I try?
        
        
### All of the Null Treatments and other subsettings have been copied from the EDA.ipynb jupyter notebook Please refer to that if you find any confusions

In [1]:
## Loading the important libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sns
import sklearn
%matplotlib inline

In [2]:
df_raw = pd.read_csv('C:/Lean_Skill_InterviewTest/data/beer-review-data-set.csv')
df_raw["user_birthdayRaw"].fillna(0,inplace= True)
df_raw["review_text"].fillna(0,inplace= True)
df_raw["user_birthdayRaw"].fillna('unknwn',inplace= True)
df_raw["user_birthdayUnix"].fillna(0,inplace= True)
df_raw["user_gender"].fillna('unknwn',inplace= True)
df_raw["user_ageInSeconds"].fillna('0',inplace= True)
df_raw["user_profileName"].fillna('unknwn',inplace= True)
df_raw.isnull().sum()
#df_raw.columns
### Reading the data and Treating all the null values in the data frame 
df_raw.describe()

,index,beer_ABV,beer_beerId,beer_brewerId,review_appearance,review_aroma,review_overall,review_palate,review_taste,review_timeUnix,user_birthdayUnix
count,37500.000000,37500.000000,37500.000000,37500.000000,37500.000000,37500.000000,37500.00000,37500.000000,37500.000000,3.750000e+04,3.750000e+04
mean,24951.887573,7.403725,21861.152027,3036.595120,3.900053,3.873240,3.88944,3.854867,3.922440,1.232794e+09,5.061995e+07
std,14434.009669,2.318145,18923.130832,5123.084675,0.588778,0.680865,0.70045,0.668068,0.716504,7.190955e+07,1.831302e+08
min,0.000000,0.100000,175.000000,1.000000,0.000000,1.000000,0.00000,1.000000,1.000000,9.262944e+08,-2.208960e+09
25%,12422.500000,5.400000,5441.000000,395.000000,3.500000,3.500000,3.50000,3.500000,3.500000,1.189194e+09,0.000000e+00
50%,24942.500000,6.900000,17538.000000,1199.000000,4.000000,4.000000,4.00000,4.000000,4.000000,1.248150e+09,0.000000e+00
75%,37416.750000,9.400000,34146.000000,1315.000000,4.500000,4.500000,4.50000,4.500000,4.500000,1.291330e+09,0.000000e+00
max,49999.000000,57.700000,77207.000000,27797.000000,5.000000,5.000000,5.00000,5.000000,5.000000,1.326267e+09,7.148988e+08


In [3]:
# Lets start with question 3
#Over here we need to find the best factor affecting the beer 
df_raw.head()

,index,beer_ABV,beer_beerId,beer_brewerId,beer_name,beer_style,review_appearance,review_aroma,review_overall,review_palate,review_taste,review_text,review_timeStruct,review_timeUnix,user_ageInSeconds,user_birthdayRaw,user_birthdayUnix,user_gender,user_profileName
0,40163,5.0,46634,14338,Chiostro,Herbed / Spiced Beer,4.0,4.0,4.0,4.0,4.0,Pours a clouded gold with a thin white head. N...,"{'min': 38, 'hour': 3, 'mday': 16, 'sec': 10, ...",1229398690,0,0,0.0,unknwn,RblWthACoz
1,8135,11.0,3003,395,Bearded Pat's Barleywine,American Barleywine,4.0,3.5,3.5,3.5,3.0,12oz bottle into 8oz snifter.\t\tDeep ruby red...,"{'min': 38, 'hour': 23, 'mday': 8, 'sec': 58, ...",1218238738,0,0,0.0,unknwn,BeerSox
2,10529,4.7,961,365,Naughty Nellie's Ale,American Pale Ale (APA),3.5,4.0,3.5,3.5,3.5,First enjoyed at the brewpub about 2 years ago...,"{'min': 7, 'hour': 18, 'mday': 26, 'sec': 2, '...",1101492422,0,0,0.0,Male,mschofield
3,44610,4.4,429,1,Pilsner Urquell,Czech Pilsener,3.0,3.0,2.5,3.0,3.0,First thing I noticed after pouring from green...,"{'min': 7, 'hour': 1, 'mday': 20, 'sec': 5, 'y...",1308532025,1.20983e+09,10-Aug-76,208508400.0,Male,molegar76
4,37062,4.4,4904,1417,Black Sheep Ale (Special),English Pale Ale,4.0,3.0,3.0,3.5,2.5,A: pours an amber with a one finger head but o...,"{'min': 51, 'hour': 6, 'mday': 12, 'sec': 48, ...",1299912708,0,0,0.0,unknwn,Brewbro000


In [4]:
### Again lets take the sum of all the reviews 
df_raw['total_rating'] = df_raw.apply(lambda x: x['review_aroma']+x['review_appearance'] + x['review_taste']+x['review_palate']+x['review_overall'], axis=1)

In [5]:
ratings_df=df_raw[['total_rating','review_aroma','review_appearance','review_taste','review_palate','review_overall']]
ratings_df['avg_ratings']=ratings_df['total_rating']/5
ratings_df

c:\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,total_rating,review_aroma,review_appearance,review_taste,review_palate,review_overall,avg_ratings
0,20.0,4.0,4.0,4.0,4.0,4.0,4.0
1,17.5,3.5,4.0,3.0,3.5,3.5,3.5
2,18.0,4.0,3.5,3.5,3.5,3.5,3.6
3,14.5,3.0,3.0,3.0,3.0,2.5,2.9
4,16.0,3.0,4.0,2.5,3.5,3.0,3.2
5,20.5,4.0,4.0,4.5,4.0,4.0,4.1
6,20.5,4.5,4.0,4.0,4.0,4.0,4.1
7,20.0,4.0,4.0,4.0,4.0,4.0,4.0
8,22.0,4.0,4.5,4.5,4.5,4.5,4.4
9,10.5,2.0,2.0,2.0,2.5,2.0,2.1


In [6]:
ratings_df.shape


(37500, 7)

In [7]:
ratings_df.corr()

,total_rating,review_aroma,review_appearance,review_taste,review_palate,review_overall,avg_ratings
total_rating,1.000000,0.831176,0.728683,0.904597,0.858382,0.865167,1.000000
review_aroma,0.831176,1.000000,0.538076,0.711809,0.608922,0.616117,0.831176
review_appearance,0.728683,0.538076,1.000000,0.531676,0.555833,0.498733,0.728683
review_taste,0.904597,0.711809,0.531676,1.000000,0.732092,0.785220,0.904597
review_palate,0.858382,0.608922,0.555833,0.732092,1.000000,0.697220,0.858382
review_overall,0.865167,0.616117,0.498733,0.785220,0.697220,1.000000,0.865167
avg_ratings,1.000000,0.831176,0.728683,0.904597,0.858382,0.865167,1.000000


### ***Answer 3***

## Based on Pearson correlation, Taste is the review attribute that is most correlated with the overall review (quality) , this will not inclued high correlation swith Total and Average ratings because they are derived variables from the the review taste feature and will always show High correlation.